In [17]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

base_path = '/kaggle/input/nitro-language-processing-2'
train_df = pd.read_csv(f'{base_path}/train_data.csv')

In [102]:
train_df

,Text,Final Labels,Id
0,@CorinaTomescu05 Încă nu ...dar am trecut prin...,non-offensive,0
1,@emosaphicbitch sau rosu ca mine,non-offensive,1
2,@DanaMinodora Ce frumoasa ești.. Arăți foarte ...,non-offensive,2
3,Din fericire în extaz!Ai dus covoareleeee?? Pu...,offensive,3
4,cand aveam vreo 5 ani credeam ca romana e sing...,non-offensive,4
...,...,...,...
39003,"@violetAndro @CorinaCg3 Așa e, știu atâtea exe...",non-offensive,39003
39004,"Directorul medical SAJ Galați, despre dispecer...",non-offensive,39004
39005,"Au ""invatat"" prostii de balta de la Satan ce e...",non-offensive,39005
39006,@_seriosul_ E o parte de adevar. Dar pana la u...,non-offensive,39006


In [18]:
import spacy

In [2]:
!python -m spacy download ro_core_news_sm
!python -m spacy download ro_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 84.5 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.5/568.5 MB 2.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_lg')


In [116]:
nlp = spacy.load("ro_core_news_lg")

In [117]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']

In [130]:
no_outliers_df = train_df[train_df['Text'].str.len() < 400]

text = no_outliers_df['Text']


tensors = []
for i, doc in enumerate(nlp.pipe(text, disable=['tagger', 'parser', 'lemmatizer', 'attribute_ruler', 'ner'], batch_size=1000)):
    if (i+1) % 1000 == 0: 
        print(i, 'of', len(text))
    tensors.append(doc.tensor)

999 of 38975
1999 of 38975
2999 of 38975
3999 of 38975
4999 of 38975
5999 of 38975
6999 of 38975
7999 of 38975
8999 of 38975
9999 of 38975
10999 of 38975
11999 of 38975
12999 of 38975
13999 of 38975
14999 of 38975
15999 of 38975
16999 of 38975
17999 of 38975
18999 of 38975
19999 of 38975
20999 of 38975
21999 of 38975
22999 of 38975
23999 of 38975
24999 of 38975
25999 of 38975
26999 of 38975
27999 of 38975
28999 of 38975
29999 of 38975
30999 of 38975
31999 of 38975
32999 of 38975
33999 of 38975
34999 of 38975
35999 of 38975
36999 of 38975
37999 of 38975


In [131]:
np.mean([t.shape[0] for t in tensors]), np.median([t.shape[0] for t in tensors])

(24.74937780628608, 20.0)

In [132]:
from sklearn.preprocessing import LabelEncoder
encoder  = LabelEncoder()
encoder.fit(no_outliers_df['Final Labels'].unique())

LabelEncoder()

In [137]:
labels = encoder.transform(no_outliers_df['Final Labels'])

In [138]:
MAX_LEN = 25
clipped_tensors = []
for t in tensors:
    new_t = t.reshape(-1)[:MAX_LEN*96]
    if len(new_t) < MAX_LEN*96:
        new_t = np.pad(new_t, (0, MAX_LEN*96 - len(new_t)), 'constant', constant_values=0)
    clipped_tensors.append(new_t)
clipped_tensors = np.array(clipped_tensors, dtype=object)

In [135]:
!pip install pytictoc

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, f1_score
from pytictoc import TicToc


splitter = StratifiedKFold(n_splits=5, random_state=42069)
for i, (train_index, test_index) in enumerate(splitter.split(clipped_tensors[:5000], labels[:5000])):
    model = LinearSVC(random_state=42069)
    
    train_X, train_y = clipped_tensors[train_index], labels[train_index]
    val_X, val_y = clipped_tensors[test_index], labels[test_index]
    
    print("split k =", i+1)
    t = TicToc()
    t.tic()
    model.fit(train_X, train_y)
    
    print("balanced acc =", balanced_accuracy_score(val_y, model.predict(val_X)))
    print("f1           =", f1_score(val_y, model.predict(val_X)))
    t.toc()

split k = 1
